In [26]:
import itertools

In [3]:
test_input = '''Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II'''

full_input = '''Valve XC has flow rate=0; tunnels lead to valves YK, AM
Valve ME has flow rate=0; tunnels lead to valves UU, SX
Valve EP has flow rate=0; tunnels lead to valves YS, QU
Valve GR has flow rate=0; tunnels lead to valves QZ, OG
Valve FA has flow rate=0; tunnels lead to valves DB, DP
Valve UJ has flow rate=0; tunnels lead to valves XN, CH
Valve QU has flow rate=0; tunnels lead to valves EP, YK
Valve OX has flow rate=19; tunnels lead to valves RI, PV
Valve VI has flow rate=0; tunnels lead to valves WI, XN
Valve IQ has flow rate=0; tunnels lead to valves QL, OG
Valve XO has flow rate=0; tunnels lead to valves GU, UI
Valve IY has flow rate=0; tunnels lead to valves VC, NT
Valve YS has flow rate=24; tunnel leads to valve EP
Valve XN has flow rate=7; tunnels lead to valves DG, UJ, VD, VI, OU
Valve AM has flow rate=6; tunnels lead to valves KA, NC, XC, TP, SI
Valve IH has flow rate=8; tunnels lead to valves TW, CH, WY, EC
Valve ZR has flow rate=18; tunnel leads to valve RI
Valve FP has flow rate=14; tunnels lead to valves DP, UF
Valve KA has flow rate=0; tunnels lead to valves VC, AM
Valve NC has flow rate=0; tunnels lead to valves UI, AM
Valve EC has flow rate=0; tunnels lead to valves IH, GU
Valve DG has flow rate=0; tunnels lead to valves AA, XN
Valve RI has flow rate=0; tunnels lead to valves OX, ZR
Valve NJ has flow rate=0; tunnels lead to valves YK, TW
Valve OG has flow rate=12; tunnels lead to valves GR, WY, IQ, UE
Valve IB has flow rate=0; tunnels lead to valves VB, UU
Valve RP has flow rate=0; tunnels lead to valves UI, OU
Valve OU has flow rate=0; tunnels lead to valves XN, RP
Valve NT has flow rate=0; tunnels lead to valves IY, AA
Valve MN has flow rate=0; tunnels lead to valves LX, VC
Valve SI has flow rate=0; tunnels lead to valves AM, AA
Valve VB has flow rate=0; tunnels lead to valves KT, IB
Valve UI has flow rate=4; tunnels lead to valves YI, XO, LX, NC, RP
Valve DL has flow rate=0; tunnels lead to valves GU, UE
Valve CH has flow rate=0; tunnels lead to valves UJ, IH
Valve WI has flow rate=0; tunnels lead to valves VI, VC
Valve GU has flow rate=11; tunnels lead to valves EC, XO, DL, SX
Valve KT has flow rate=17; tunnels lead to valves PV, VB
Valve TW has flow rate=0; tunnels lead to valves IH, NJ
Valve UE has flow rate=0; tunnels lead to valves DL, OG
Valve PV has flow rate=0; tunnels lead to valves KT, OX
Valve DP has flow rate=0; tunnels lead to valves FP, FA
Valve TP has flow rate=0; tunnels lead to valves VD, AM
Valve YI has flow rate=0; tunnels lead to valves AA, UI
Valve LX has flow rate=0; tunnels lead to valves UI, MN
Valve QZ has flow rate=0; tunnels lead to valves GR, UU
Valve DB has flow rate=23; tunnel leads to valve FA
Valve SX has flow rate=0; tunnels lead to valves ME, GU
Valve QL has flow rate=0; tunnels lead to valves AA, IQ
Valve YK has flow rate=16; tunnels lead to valves NJ, XC, QU
Valve VC has flow rate=5; tunnels lead to valves UF, KA, WI, IY, MN
Valve VD has flow rate=0; tunnels lead to valves TP, XN
Valve WY has flow rate=0; tunnels lead to valves IH, OG
Valve AA has flow rate=0; tunnels lead to valves YI, DG, QL, NT, SI
Valve UF has flow rate=0; tunnels lead to valves VC, FP
Valve UU has flow rate=15; tunnels lead to valves QZ, IB, ME'''

In [83]:
def parse_input(input_str):
    graph = {}
    rates = {}
    for line in input_str.split('\n'):

        substrs = line.split()
        valve = substrs[1]
        rate = substrs[4]
        rate = int(rate[5:][:-1])
        if rate:
            rates[valve] = rate
            print(valve, rate)
        graph[valve] = [v.strip(',') for v in substrs[9:]]
        
    return graph, rates


def dijkstra(graph, start):
    
    distances = {}
    previous = {}
    unvisited = set()     
    for vertex in graph:
        distances[vertex] = len(graph)
        previous[vertex] = None
        unvisited.add(vertex)
    distances[start] = 0   
    
    while len(unvisited):
        minimum = len(graph)
        for vertex in unvisited:
            if distances[vertex] < minimum:
                closest = vertex
                minimum = distances[vertex]
        unvisited.remove(closest)

        for neighbour in graph[closest]:
            if neighbour not in unvisited:
                continue
            if distances[closest] + 1 < distances[neighbour]:
                distances[neighbour] = distances[closest] + 1
                previous[neighbour] = closest

    return distances

def calc_distances(graph, valves):
    distances = {}
    for value in valves:
        distances[value] = dijkstra(graph, value)   
    return distances

def calc_time(path):
    time = 0
    previous = 'AA'
    for valve in path:
        time += 1
        time += distances[previous][valve]
        previous = valve
    return time

In [133]:
# part 1
input_str = test_input
input_str = full_input
graph, rates = parse_input(input_str)
openable = list(rates.keys())
# print(openable)
distances = calc_distances(graph, ['AA'] + openable)
# print(distances)

paths = [[valve] for valve in openable]
new_paths = paths

while True:
    old_paths = new_paths
    new_paths = []
    for path in old_paths:
        remaining = set(openable) - set(path)
        for valve in remaining:
            new_path = path + [valve]
            time = calc_time(new_path)
            if time <= 30:
                new_paths.append(new_path)

    for path in new_paths:
        paths.append(path)
        if path[:-1] in paths:
            paths.remove(path[:-1])

    if not new_paths:
        break

print(len(paths))

max_released = 0
for path in paths:

    released = 0
    current = 'AA'
    target = None
    open_valves = []
    for time in range(30):
        for valve in open_valves:
            released += rates[valve]

        if target is None:
            if not path:
                continue
            target = path.pop(0)
            distance = distances[current][target] - 1
        elif distance == 0:
            open_valves.append(target)
            current = target
            target = None
        else:
            distance += -1

    if released > max_released:
        max_released  = released
    
print('Max released:', max_released)

OX 19
YS 24
XN 7
AM 6
IH 8
ZR 18
FP 14
OG 12
UI 4
GU 11
KT 17
DB 23
YK 16
VC 5
UU 15
134688
Max released: 1460


In [235]:
# part 2

input_str = test_input
input_str = full_input
graph, rates = parse_input(input_str)
openable = list(rates.keys())
distances = calc_distances(graph, ['AA'] + openable)
   
paths = [[valve] for valve in openable]
new_paths = paths

while True:
    old_paths = new_paths
    new_paths = []
    for path in old_paths:
        remaining = set(openable) - set(path)
        for valve in remaining:
            new_path = path + [valve]
            time = calc_time(new_path)
            if time <= 26:
                new_paths.append(new_path)

    for path in new_paths:
        paths.append(path)
        if path[:-1] in paths:
            paths.remove(path[:-1])

    if not new_paths:
        break   

print(len(paths))

human_paths = paths
paths = []
for human_path in human_paths:
    remaining = set(openable) - set(human_path)
    new_paths = [[valve] for valve in remaining]
    elephant_paths = []
    while True:
        old_paths = new_paths
        new_paths = []
        for path in old_paths:
            remaining = set(openable) - set(path) - set(human_path)
            for valve in remaining:
                new_path = path + [valve]
                time = calc_time(new_path)
                if time <= 26:
                    new_paths.append(new_path)
                    
        for path in new_paths:
            elephant_paths.append(path)
            if path[:-1] in elephant_paths:
                elephant_paths.remove(path[:-1])

        if not new_paths:
            break
            
    for elephant_path in elephant_paths:
        paths.append((human_path, elephant_path))
        
        
print(len(paths))

max_released = 0
for path in paths:
    human_path, elephant_path = path
    released = 0
    human_current = 'AA'
    human_target = None
    elephant_current = 'AA'
    elephant_target = None
    open_valves = []
    for time in range(26):
        for valve in open_valves:
            released += rates[valve]

        if human_target is None:
            if human_path:
                human_target = human_path.pop(0)
                human_distance = distances[human_current][human_target] - 1
        elif human_distance == 0:
            open_valves.append(human_target)
            human_current = human_target
            human_target = None
        else:
            human_distance += -1

        if elephant_target is None:
            if elephant_path:
                elephant_target = elephant_path.pop(0)
                elephant_distance = distances[elephant_current][elephant_target] - 1
        elif elephant_distance == 0:
            open_valves.append(elephant_target)
            elephant_current = elephant_target
            elephant_target = None
        else:
            elephant_distance += -1
            
    if released > max_released:
        max_released  = released
    
print('Max released:', max_released)

OX 19
YS 24
XN 7
AM 6
IH 8
ZR 18
FP 14
OG 12
UI 4
GU 11
KT 17
DB 23
YK 16
VC 5
UU 15
35858
43408891
Max released: 1682


In [198]:
(1, 2)[::-1]

(2, 1)

In [146]:
input_str = test_input
graph, openable = parse_input(input_str)
distances = calc_distances(graph, ['AA'] + list(openable.keys()))
print()

human_path = ['JJ', 'BB', 'CC']
elephant_path = ['DD', 'HH', 'EE']

# elephant_path = ['JJ', 'BB', 'CC']
# human_path = ['DD', 'HH', 'EE']

released = 0
human_current = 'AA'
human_target = None
elephant_current = 'AA'
elephant_target = None
open_valves = []
for time in range(26):
    for valve in open_valves:
        released += openable[valve]
    print(time + 1, 'open', ', '.join(open_valves))    
        

    if human_target is None:
        if human_path:
            human_target = human_path.pop(0)
            human_distance = distances[human_current][human_target] - 1
            print(time + 1, 'you', human_target, human_distance + 1)
    elif human_distance == 0:
        open_valves.append(human_target)
        human_current = human_target
        print(time + 1, 'you opening', human_target)
        human_target = None
    else:
        human_distance += -1
        print(time + 1, 'you', human_target, human_distance + 1)
        
    if elephant_target is None:
        if elephant_path:
            elephant_target = elephant_path.pop(0)
            elephant_distance = distances[elephant_current][elephant_target] - 1
            print(time + 1, 'elephant', elephant_target, elephant_distance + 1)
    elif elephant_distance == 0:
        open_valves.append(elephant_target)
        elephant_current = elephant_target
        print(time + 1, 'elephant opening', elephant_target)
        elephant_target = None
    else:
        elephant_distance += -1
        print(time + 1, 'elephant', elephant_target, elephant_distance + 1)
        
        
    print()
            
print(released)

BB 13
CC 2
DD 20
EE 3
HH 22
JJ 21

1 open 
1 you DD 1
1 elephant JJ 2

2 open 
2 you opening DD
2 elephant JJ 1

3 open DD
3 you HH 4
3 elephant opening JJ

4 open DD, JJ
4 you HH 3
4 elephant BB 3

5 open DD, JJ
5 you HH 2
5 elephant BB 2

6 open DD, JJ
6 you HH 1
6 elephant BB 1

7 open DD, JJ
7 you opening HH
7 elephant opening BB

8 open DD, JJ, HH, BB
8 you EE 3
8 elephant CC 1

9 open DD, JJ, HH, BB
9 you EE 2
9 elephant opening CC

10 open DD, JJ, HH, BB, CC
10 you EE 1

11 open DD, JJ, HH, BB, CC
11 you opening EE

12 open DD, JJ, HH, BB, CC, EE

13 open DD, JJ, HH, BB, CC, EE

14 open DD, JJ, HH, BB, CC, EE

15 open DD, JJ, HH, BB, CC, EE

16 open DD, JJ, HH, BB, CC, EE

17 open DD, JJ, HH, BB, CC, EE

18 open DD, JJ, HH, BB, CC, EE

19 open DD, JJ, HH, BB, CC, EE

20 open DD, JJ, HH, BB, CC, EE

21 open DD, JJ, HH, BB, CC, EE

22 open DD, JJ, HH, BB, CC, EE

23 open DD, JJ, HH, BB, CC, EE

24 open DD, JJ, HH, BB, CC, EE

25 open DD, JJ, HH, BB, CC, EE

26 open DD, JJ, HH, BB,

In [138]:
input_str = test_input
# input_str = full_input
graph, openable = parse_input(input_str)
# print(graph)
print(len(openable))
distances = calc_distances(graph, ['AA'] + list(openable.keys()))
print(distances)

path = ['DD', 'BB', 'JJ', 'HH', 'EE', 'CC']

time = 0
previous = 'AA'
for valve in path:
    time += 1
    time += distances[previous][valve]
    previous = valve
    
print(time)



released = 0
current = 'AA'
target = None
open_valves = []
for time in range(30):
    for valve in open_valves:
        released += openable[valve]
    print(time + 1, 'open', ','.join(open_valves))    
        
    if target is None:
        if not path:
            continue
        target = path.pop(0)
        distance = distances[current][target] - 1
        print(time + 1, target, distance + 1)
    elif distance == 0:
        open_valves.append(target)
        current = target
        print(time + 1, 'opening', target)
        target = None
    else:
        distance += -1
        print(time + 1, target, distance + 1)
        
    print()
            
print(released)   
    

BB 13
CC 2
DD 20
EE 3
HH 22
JJ 21
6
{'AA': {'AA': 0, 'BB': 1, 'CC': 2, 'DD': 1, 'EE': 2, 'FF': 3, 'GG': 4, 'HH': 5, 'II': 1, 'JJ': 2}, 'BB': {'AA': 1, 'BB': 0, 'CC': 1, 'DD': 2, 'EE': 3, 'FF': 4, 'GG': 5, 'HH': 6, 'II': 2, 'JJ': 3}, 'CC': {'AA': 2, 'BB': 1, 'CC': 0, 'DD': 1, 'EE': 2, 'FF': 3, 'GG': 4, 'HH': 5, 'II': 3, 'JJ': 4}, 'DD': {'AA': 1, 'BB': 2, 'CC': 1, 'DD': 0, 'EE': 1, 'FF': 2, 'GG': 3, 'HH': 4, 'II': 2, 'JJ': 3}, 'EE': {'AA': 2, 'BB': 3, 'CC': 2, 'DD': 1, 'EE': 0, 'FF': 1, 'GG': 2, 'HH': 3, 'II': 3, 'JJ': 4}, 'HH': {'AA': 5, 'BB': 6, 'CC': 5, 'DD': 4, 'EE': 3, 'FF': 2, 'GG': 1, 'HH': 0, 'II': 6, 'JJ': 7}, 'JJ': {'AA': 2, 'BB': 3, 'CC': 4, 'DD': 3, 'EE': 4, 'FF': 5, 'GG': 6, 'HH': 7, 'II': 1, 'JJ': 0}}
24
1 open 
1 DD 1

2 open 
2 opening DD

3 open DD
3 BB 2

4 open DD
4 BB 1

5 open DD
5 opening BB

6 open DD,BB
6 JJ 3

7 open DD,BB
7 JJ 2

8 open DD,BB
8 JJ 1

9 open DD,BB
9 opening JJ

10 open DD,BB,JJ
10 HH 7

11 open DD,BB,JJ
11 HH 6

12 open DD,BB,JJ
12 HH 5

13 open 